## Answer 1

In [1]:
# Download GDPC1.csv
# download manually

In [2]:
import pandas as pd

# Read dataset
df = pd.read_csv("GDPC1.csv")

# Convert DATE column to datetime if it's not already in datetime format
df['DATE'] = pd.to_datetime(df['DATE'])

# Shift GDPC1 column by 4 rows to get value from one year ago
df['GDPC1_one_year_ago'] = df['GDPC1'].shift(4)

# Calculate YoY growth rate
df['YoY_growth'] = df['GDPC1'] / df['GDPC1_one_year_ago'] - 1

# Filter dataframe to include only rows from 2023
df_2023 = df[df['DATE'].dt.year == 2023]

# Calculate average YoY growth rate for 2023
average_YoY_growth_2023 = df_2023['YoY_growth'].mean()

# Round to 1 decimal point
average_YoY_growth_2023_rounded = round(average_YoY_growth_2023 * 100, 1)

print("Average YoY growth rate in 2023: {}%".format(average_YoY_growth_2023_rounded))

Average YoY growth rate in 2023: 2.5%


## Answer 2

In [3]:
import pandas as pd

# Download DGS2 and DGS10 interest rates series
dgs2_url  = "https://fred.stlouisfed.org/series/DGS2/downloaddata/DGS2.csv"
dgs10_url = "https://fred.stlouisfed.org/series/DGS10/downloaddata/DGS10.csv"

In [4]:
# Read data into pandas dataframes
dgs2 = pd.read_csv(dgs2_url, parse_dates=['DATE'], index_col='DATE')
dgs10 = pd.read_csv(dgs10_url, parse_dates=['DATE'], index_col='DATE')

In [5]:
# Join the dataframes on date
df = dgs10.join(dgs2, lsuffix='_dgs10', rsuffix='_dgs2')

In [6]:
# Change empty or Nan (null) into 0 (zero)
df.fillna(0, inplace=True)

In [ ]:
# Filter DataFrame to only contain rows where all values are numeric
df = df.apply(lambda col: pd.to_numeric(col, errors='coerce')).dropna()

In [ ]:
df

In [ ]:
# Current data types
df.dtypes

In [ ]:
# Calculate the difference dgs10 - dgs2 daily
df['difference'] = df['VALUE_dgs10'] - df['VALUE_dgs2']

In [ ]:
df['difference'] 

In [ ]:
# Filter data since year 2000
df_since_2000 = df[df.index >= '2000-01-01']

# Find the minimum value of the difference
min_difference = df_since_2000['difference'].min()

# Round to 1 decimal point
min_difference_rounded = round(min_difference, 1)

print("Minimum value of (dgs10 - dgs2) since 2000: {}".format(min_difference_rounded))

## Answer 3

In [ ]:
import yfinance as yf

# Download data for S&P 500 (^GSPC) and IPC Mexico (^MXX) indexes
sp500 = yf.download('^GSPC', start='2019-04-09', end='2024-04-09')['Adj Close']
ipc_mexico = yf.download('^MXX', start='2019-04-09', end='2024-04-09')['Adj Close']

In [ ]:
# Calculate the 5-year growth for both indexes
sp500_growth = (sp500.iloc[-1] / sp500.iloc[0] - 1) * 100
ipc_mexico_growth = (ipc_mexico.iloc[-1] / ipc_mexico.iloc[0] - 1) * 100

In [ ]:
# Determine which index has the highest growth
if sp500_growth > ipc_mexico_growth:
    largest_growth = round(sp500_growth)
    index_name = "S&P 500"
else:
    largest_growth = round(ipc_mexico_growth)
    index_name = "IPC Mexico"

print("The index with the largest 5-year growth is {} with a growth of {}%.".format(index_name, largest_growth))

## Answer 4

In [ ]:
import yfinance as yf

# List of selected stocks
stocks = ['2222.SR', 'BRK-B', 'AAPL', 'MSFT', 'GOOG', 'JPM']

# Download 2023 daily OHLCV data for each stock
stock_data = {}
for stock in stocks:
    stock_data[stock] = yf.download(stock, start='2023-01-01', end='2023-12-31')

# Calculate maximum-minimum "Adj.Close" price for each stock and divide by the maximum "Adj.Close" value
max_range_ratio = 0
max_range_stock = None
for stock, data in stock_data.items():
    max_adj_close = data['Adj Close'].max()
    min_adj_close = data['Adj Close'].min()
    range_ratio = (max_adj_close - min_adj_close) / max_adj_close
    if range_ratio > max_range_ratio:
        max_range_ratio = range_ratio
        max_range_stock = stock

# Round the result to two decimal places
max_range_ratio_rounded = round(max_range_ratio, 2)

print("The largest range ratio in 2023 is {} for stock {}.".format(max_range_ratio_rounded, max_range_stock))


## Answer 5

In [ ]:
import yfinance as yf

# List of selected stocks
stocks = ['2222.SR', 'BRK-B', 'AAPL', 'MSFT', 'GOOG', 'JPM']

# Dictionary to store dividends paid by each stock
dividends_paid = {}

# Loop through each stock and download dividends paid in 2023
for stock in stocks:
    stock_info = yf.Ticker(stock)
    dividends = stock_info.dividends
    dividends_2023 = dividends.loc['2023']
    total_dividends = dividends_2023.sum()
    dividends_paid[stock] = total_dividends

# Dictionary to store the closing price of each stock at the last trading day of the year
closing_prices = {}

# Loop through each stock and download the closing price at the last trading day of 2023
for stock in stocks:
    stock_data = yf.download(stock, end='2023-12-31')
    closing_prices[stock] = stock_data['Adj Close'][-1]

# Calculate dividend yield for each stock
dividend_yield = {}
for stock in stocks:
    if closing_prices[stock] != 0:  # Avoid division by zero
        dividend_yield[stock] = (dividends_paid[stock] / closing_prices[stock]) * 100
    else:
        dividend_yield[stock] = 0

# Find the largest dividend yield
largest_dividend_yield = max(dividend_yield.values())

# Round to 1 decimal point
largest_dividend_yield_rounded = round(largest_dividend_yield, 1)

print("The largest dividend yield in 2023 is {}%.".format(largest_dividend_yield_rounded))
